# DM (device manufacturer) connects to music (device) and assign them ownership - Music

## 3. Initialize Music

In [1]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"
API_KEY = "bob_api_123456789"

agent_controller = AriesAgentController(admin_url=ADMIN_URL, api_key=API_KEY)

agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener], defaults=True)


######################
# Remember to update #
######################

invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'ad9330d4-0375-407c-a5a1-82c448df7641', 'label': 'Alice', 'serviceEndpoint': 'http://192.168.65.3:8020', 'recipientKeys': ['E9Tkz2maMyr5FsQyZTxEMVeNYhCk1kHXAvc8cjhVgRzJ']}


# Receive Invitation
response = await agent_controller.connections.accept_connection(invitation)
# Print out accepted Invite and Alice's connection ID
print("Connection", response)
alice_id = response["connection_id"]


IPython autoawait is `on`, and set to use `asyncio`
Subscribing too: connections
Connection Handler Called
Connection 8c0f93ff-0574-4eb9-96e8-37d5618d7f13 in State invitation
Connection Handler Called
Connection 8c0f93ff-0574-4eb9-96e8-37d5618d7f13 in State request
Connection {'created_at': '2023-01-02 18:50:31.585805Z', 'invitation_key': 'E9Tkz2maMyr5FsQyZTxEMVeNYhCk1kHXAvc8cjhVgRzJ', 'routing_state': 'none', 'their_label': 'Alice', 'request_id': '3167f33a-b862-4a0d-a14b-8f6c2c4dfce6', 'connection_id': '8c0f93ff-0574-4eb9-96e8-37d5618d7f13', 'their_role': 'inviter', 'rfc23_state': 'request-sent', 'accept': 'manual', 'invitation_mode': 'once', 'state': 'request', 'my_did': 'KmhihRYddnwfZ6tk1rgrjZ', 'updated_at': '2023-01-02 18:50:31.667972Z'}


In [2]:
# Print connection list
connection = await agent_controller.connections.get_connection(alice_id)
print("Alice AGENT CONNECTION")
print(connection)
print("State:", connection["state"])

Alice AGENT CONNECTION
{'created_at': '2023-01-02 18:50:31.585805Z', 'invitation_key': 'E9Tkz2maMyr5FsQyZTxEMVeNYhCk1kHXAvc8cjhVgRzJ', 'routing_state': 'none', 'their_label': 'Alice', 'request_id': '3167f33a-b862-4a0d-a14b-8f6c2c4dfce6', 'connection_id': '8c0f93ff-0574-4eb9-96e8-37d5618d7f13', 'their_role': 'inviter', 'rfc23_state': 'request-sent', 'accept': 'manual', 'invitation_mode': 'once', 'state': 'request', 'my_did': 'KmhihRYddnwfZ6tk1rgrjZ', 'updated_at': '2023-01-02 18:50:31.667972Z'}
State: request
Connection Handler Called
Connection 8c0f93ff-0574-4eb9-96e8-37d5618d7f13 in State response
Connection Handler Called
Connection 8c0f93ff-0574-4eb9-96e8-37d5618d7f13 in State active


# Give MUSIC the VC of ownership and origin - DM

## 3. Credential Exchange!

In [3]:
response = await agent_controller.issuer.get_records()
results = response["results"]
if len(results) == 0:
    print("You need to first send a credential from the issuer notebook (Alice)")
else:
    cred_record = results[0]
    cred_ex_id = cred_record['credential_exchange_id']
    state = cred_record['state']
    role = cred_record['role']
    attributes = results[0]['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")

Credential exchange 21e136bb-c35b-4a14-84b4-9851e9323fe5, role: holder, state: offer_received
Being offered: [{'name': 'author', 'value': 'PQRXDxdGqQGSZ8z69p4xZP'}, {'name': 'name', 'value': 'balabababa'}, {'name': 'time', 'value': '01-02-2023, 18:54:19'}, {'name': 'original_author', 'value': 'PQRXDxdGqQGSZ8z69p4xZP'}]


In [4]:
# Request credential from issuer
record = await agent_controller.issuer.send_request_for_record(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange 21e136bb-c35b-4a14-84b4-9851e9323fe5, role: holder, state: request_sent


In [5]:
record = await agent_controller.issuer.get_record_by_id(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange 21e136bb-c35b-4a14-84b4-9851e9323fe5, role: holder, state: credential_received


In [6]:
# Store credential
response = await agent_controller.issuer.store_credential(cred_ex_id, "My OM Credential")
state = response['state']
role = response['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange 21e136bb-c35b-4a14-84b4-9851e9323fe5, role: holder, state: credential_acked


## Remove connection !!!

In [9]:
remove = await agent_controller.connections.remove_connection(alice_id)
print(remove)
response = await agent_controller.connections.get_connections()
print(response)

{}
{'results': [{'their_label': 'Alice', 'accept': 'manual', 'state': 'request', 'connection_id': '521d9010-2f24-4a6b-9f48-c369db186f0b', 'invitation_key': 'GC6DHU7HHN4YVKoAHqrNg3VkpTyWUTpY7zKmpZSYvW1z', 'routing_state': 'none', 'created_at': '2023-01-02 13:20:47.668037Z', 'updated_at': '2023-01-02 13:20:47.728024Z', 'their_role': 'inviter', 'request_id': '18a7c875-3cf3-450e-92b6-850ddad8d0b8', 'invitation_mode': 'once', 'rfc23_state': 'request-sent', 'my_did': '2sTtp6vonk2F6gGjQELqFD'}, {'their_label': 'Alice', 'accept': 'manual', 'state': 'response', 'connection_id': '663ddab5-2135-478a-b7f1-3d7dfebe22ad', 'invitation_key': 'HHTzqiXeQey1NmZCZRkGMbT8KejyxGna2Su3gX3kHYDw', 'routing_state': 'none', 'created_at': '2023-01-02 13:21:20.734655Z', 'updated_at': '2023-01-02 13:21:38.481793Z', 'their_role': 'inviter', 'their_did': 'VWHUDyJon3XHkwdGiF3uQ9', 'request_id': '2f6713ee-3a34-44c8-b262-d0f9e955037a', 'invitation_mode': 'once', 'rfc23_state': 'response-received', 'my_did': '7vbdgKJzN44

In [40]:
response = await agent_controller.terminate()
print(response)

None
